# GitOps Flow with MLRun

In [ ]:
from src import create_and_set_project

### Setup Project

In [ ]:
# ENV_FILE = "mlrun.env" # update to your .env file
ENV_FILE = "us-cst-353.env" # update to your .env file

In [ ]:
project = create_and_set_project(git_source="v3io:///bigdata/liveops.zip", env_file=ENV_FILE)

### Run Training Pipeline

In [ ]:
run_id = project.run(name="train", dirty=True, watch=True)

### Run Deployment Pipeline

In [ ]:
run_id = project.run(
    name="deploy",
    arguments={"model_path": "store://models/gitops-flow/model#0:latest"},
    dirty=True,
    watch=True
)

### Invoke Model

In [ ]:
serving_fn = project.get_function("serving")
serving_fn.invoke('/v2/models/model/infer', {"inputs" : [[6.1, 2.8, 4.7, 1.2], [5.7, 3.8, 1.7, 0.3], [7.7, 2.6, 6.9, 2.3]]})

### Simulate Traffic

MLRun real-time drift detection requires 10,000 events

In [ ]:
# Get serving function
serving_fn = project.get_function("serving")
model_name = "model"

# Get test set
df = project.get_artifact("train_test_set").to_dataitem().as_df()
display(df.head())

# Transform to list
# Get training set as list
iris_data = df.drop("label", axis=1).to_dict(orient="split")["data"]
iris_data[:5]

In [ ]:
import json
import logging
from random import choice

from tqdm.notebook import tqdm

# Suppress print messages
logging.getLogger(name="mlrun").setLevel(logging.WARNING)

# Simulate traffic using random elements from training set
for i in tqdm(range(12_000)):
    data_point = choice(iris_data)
    serving_fn.invoke(
        f"v2/models/{model_name}/infer", json.dumps({"inputs": [data_point]})
    )

# Resume normal logging
logging.getLogger(name="mlrun").setLevel(logging.INFO)

### Simulate Drift Event

In [ ]:
import json
import v3io.dataplane

In [ ]:
v3io_client = v3io.dataplane.Client()

In [ ]:
drift_event = {'endpoint_id': '364ccdd1c2f6627ddfb523b16f53ae2b62e56ee8', 'drift_status': 'POSSIBLE_DRIFT', 'drift_measure': 0.87}

In [ ]:
v3io_client.stream.put_records(
    container='users',
    stream_path=f"pipelines/{project.name}/model-endpoints/log_stream",
    records=[{'data': json.dumps(drift_event), 'shard_id': 0}]
)